In [1]:
# Import necessary modules
import py_ncdiff
import os
import numpy as np

# Setup logging so info / debug -> stdout instead of stderr
py_ncdiff.init_logging()

In [2]:
#file1 = os.path.join(os.path.expanduser('~/'),'MARBL_data','POP_hist.nc')
#file1 = os.path.join(os.path.sep,'sshfs','glade','scratch','baselines','single_run','C1850ECO.T62_g37.ciso.trunk_20180716.pop.h.0001-01-03.nc')
file1 = os.path.join(os.path.expanduser('~/'),'tmp','C1850ECO.T62_g37.ciso.trunk_20180705.pop.h.0001-01-03.nc')
file2 = os.path.join(os.path.expanduser('~/'),'tmp','C1850ECO.T62_g37.ciso.trunk_20180716.pop.h.0001-01-03.nc')
#bad_file = os.path.expanduser('~/')+'MARBL_data/POP_hist2.nc'

test = py_ncdiff.netCDF_comp_class(file1, file2,quiet=True)
#print("---")
#bad_test = py_ncdiff.netCDF_comp_class(file1, bad_file)

Comparing /Users/mlevy/tmp/C1850ECO.T62_g37.ciso.trunk_20180705.pop.h.0001-01-03.nc and /Users/mlevy/tmp/C1850ECO.T62_g37.ciso.trunk_20180716.pop.h.0001-01-03.nc
-----
Baseline modified: Wed Jul 25 12:50:19 2018
New file modified: Wed Jul 25 12:50:53 2018
-----


In [3]:
test.compare_variable_names()
test.compare_variable_type_and_dims()
test.compare_metadata()
test.compare_values()
err_cnt = test.parse_results()

(1) Compare Variable Names: FAIL (1334/1431 pass)
(2) Compare Variable Types and Dimensions: PASS
(3) Compare Metadata: PASS
(4) Compare Values: FAIL (257/1334 pass)


In [4]:
test.diff_vars

[u'DIA_IMPVF_diatChl',
 u'tend_zint_100m_diazP',
 u'DIA_IMPVF_spCaCO3',
 u'WT_spCaCO3',
 u'graze_sp_doc',
 u'Jint_100m_DI13C',
 u'popToSed',
 u'graze_diat_poc_zint',
 u'zsatcalc',
 u'tend_zint_100m_diazC',
 u'HDIFE_spCaCO3',
 u'graze_diaz_zoo',
 u'HDIFB_Lig',
 u'HDIFN_diatFe',
 u'HDIFB_DIC_ALT_CO2',
 u'diat_Fe_lim_Cweight_avg_100m',
 u'QSW_HTP',
 u'FvICE_DI14C',
 u'UET',
 u'DOCr_remin_zint_100m',
 u'CISO_Ca14CO3_FLUX_IN',
 u'spCa14CO3',
 u'diaz_loss_doc_zint',
 u'Jint_100m_spChl',
 u'sp_loss_doc_zint_100m',
 u'VN_diat13C',
 u'J_DO13Ctot',
 u'CISO_FG_14CO2',
 u'diaz_Fe_lim_surf',
 u'Jint_100m_spCa14CO3',
 u'J_DONr',
 u'photoC_diaz_zint_100m',
 u'diazFe',
 u'P_iron_FLUX_100m',
 u'Jint_ALK',
 u'Jint_DOPr',
 u'DIA_IMPVF_DIC',
 u'graze_diat',
 u'spC_zint_100m',
 u'CISO_Ca13CO3_REMIN',
 u'VDC_T',
 u'graze_diat_zoo',
 u'Jint_100m_DI14C',
 u'graze_diaz',
 u'HDIFE_diatC',
 u'ALK',
 u'photoFe_diaz',
 u'CISO_photo13C_diaz',
 u'photoFe_diat',
 u'CISO_photo13C_diat',
 u'CaCO3_form_zint',
 u'photoC_